In [ ]:
import numpy as np

In [ ]:
# Dictionary : Turn it into a set
vocab = set([line.rstrip() for line in open("document/vocab.txt")])

In [ ]:
def generate_candidates(word):
    # generate 1 word
    # 1.insert 2.delete 3.replace
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i],word[i:]) for i in range(len(word)+1)]
    # 1.insert operation
    inserts = [L+c+R for L,R in splits for c in letters]
    # 2.delete operation
    deletes = [L+R[1:] for L,R in splits if R]
    # 3.replace operation
    replaces = [L+c+R[1:] for L,R in splits if R for c in letters]

    candidates = set(inserts+deletes+replaces)

    return [word for word in candidates if word in vocab]

In [1]:
from nltk.corpus import reuters
# read the corpus
categories = reuters.categories()
corpus = reuters.sents(categories=categories)

In [3]:
# construct the language model : using bi-gram
term_count = {}
bigram_count = {}
for doc in corpus:
    # preprocess the first one
    doc = ['<s>'] + doc
    for i in range(0, len(doc)-1):
        # bigram:[i,i+1]
        term = doc[i]
        bigram = doc[i:i+2]

        if term in term_count:
            term_count[term] += 1
        else:
            term_count[term] = 1

        bigram = ' '.join(bigram)
        if bigram in bigram_count:
            bigram_count[bigram] += 1
        else:
            bigram_count[bigram] = 1


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\28224/nltk_data'
    - 'D:\\softwareother\\Anaconda3\\envs\\tensorflow\\nltk_data'
    - 'D:\\softwareother\\Anaconda3\\envs\\tensorflow\\share\\nltk_data'
    - 'D:\\softwareother\\Anaconda3\\envs\\tensorflow\\lib\\nltk_data'
    - 'C:\\Users\\28224\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [5]:
# channel probability
channel_prob = {}
for line in open('document/spell-errors.txt'):
    items = line.split(":")
    correct = items[0].strip()
    mistakes = [item.strip() for item in items[1].strip().split(",")]
    channel_prob[correct] = {}
    for mis in mistakes:
        channel_prob[correct][mis] = 1.0/len(mistakes)



In [ ]:
V = len(term_count.keys())

file = open('document/testdata.txt','r')
for line in file:
    items = line.rstrip().split('\t')
    line = items[2].split()
    for word in line:
        candidates = generate_candidates(word)
        probs = []
        # return the most possible one
        for candi in candidates:
            prob = 0
            # a.channel
            if candi in channel_prob and word in channel_prob[candi]:
                prob += np.log(channel_prob[candi][word])
            else:
                prob += np.log(0.0001)

            # calculate the possibility of language model:
            idx = items[2].index(word) + 1
            if items[2][idx - 1] in bigram_count and candi in bigram_count[items[2][idx-1]]:
                prob += np.log((bigram_count[items[2][idx-1]][candi] + 1.0) / (term_count[bigram_count[items[2][idx-1]]] + V))
            else:
                prob += np.log(1.0 / V)

        probs.append(prob)
    max_idx = probs.index(max(probs))
    print(word,candidates[max_idx])